<center>
    <img src="https://cds.cern.ch/record/1306150/files/cmsLogo_image.jpg" width="300" alt="cognitiveclass.ai logo"  />
    
</center>

# Pile Up Study at the CMS experiment in 2018

### importing libraries and classes


In [1]:
from __future__ import annotations
import uproot
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import plotly
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt


from plotly.subplots import make_subplots
from typing import overload
from abc import ABC, abstractmethod
import sympy
from sympy.integrals.transforms import laplace_transform
import random
import math
from tqdm.auto import tqdm, trange
from concurrent.futures import ThreadPoolExecutor
import scipy.stats as stats
from scipy.optimize import curve_fit


from Class1 import parametricValue

    
from Class1 import fixedValue


from Class1 import parametricValueNumpy


from Class1 import parametricValueSympy


#these are important for me

from Class1 import binning


from Class1 import discretepdf


from Class1 import NLL


from Class1 import lumiDist


from Class1 import lumiDistSympyLaplace


from Class1 import lumiDistNumpy


from Class1 import SimulationVSPu


In [2]:
from Class1 import RunInfo


## choosing Runs and Eras

In [3]:
from allInfo import GetAllRuns, GetAllEras

year =2018
GetAllEras(year)

['1', 'B', '5', 'A']

In [4]:
year =2018
eras = ['A' , 'B']
# for era in GetAllEras(year):
for era in eras:
    allRuns = sorted( list(set( GetAllRuns(year , era) ) ) )
    print(era, ':' , allRuns)

A : [315257, 315259, 315264, 315265, 315267, 315270, 315322, 315339, 315357, 315361, 315363, 315366, 315420, 315488, 315489, 315490, 315506, 315510, 315512, 315543, 315555, 315556, 315557, 315640, 315641, 315642, 315644, 315645, 315646, 315647, 315648, 315689, 315690, 315702, 315703, 315704, 315705, 315713, 315721, 315741, 315764, 315770, 315784, 315785, 315786, 315790, 315800, 315801, 315840, 315973, 315974, 316058, 316059, 316060, 316061, 316062, 316082, 316110, 316111, 316113, 316114, 316153, 316186, 316187, 316199, 316200, 316201, 316202, 316216, 316217, 316218, 316219, 316239, 316240, 316241, 316271, 316361, 316362, 316363, 316377, 316378, 316379, 316380, 316455, 316457, 316469, 316470, 316472, 316505, 316569, 316590, 316613, 316615, 316666, 316667, 316700, 316701, 316702, 316715, 316716, 316717, 316718, 316719, 316720, 316721, 316722, 316723, 316758, 316766, 316876, 316877, 316879, 316928, 316985, 316993, 316994, 316995]
B : [317080, 317087, 317089, 317182, 317212, 317213, 317279

In [5]:
A_allRuns = sorted( list(set( GetAllRuns(2018 , 'A') ) ) )
B_allRuns = sorted( list(set( GetAllRuns(2018 , 'B') ) ) )
print('era A has {0} runs'.format(len(A_allRuns)))
print('era B has {0} runs'.format(len(B_allRuns)))

era A has 117 runs
era B has 43 runs


In [6]:
year , era = 2018 , 'A'
allRuns = sorted( list(set( GetAllRuns(year , era) ) ) )
allRuns = allRuns[22:24]
allRuns

[315557, 315640]

In [7]:
allRuns = sorted( list(set( GetAllRuns(year , era) ) ) )
Runs = allRuns[23:28]
if len(Runs) == 1:
    ind = allRuns.index(Runs[0])
    runName = 'RunNo:{0}'.format(ind+1)
else:
    runName = 'nRuns={0}'.format(len(Runs))
runName

'nRuns=5'

## Run Cell

In [8]:
web1 = '/eos/user/c/cmstandi/SWAN_projects/Nima/PUFit' # source directory
web2 = '/eos/user/c/cmstandi/www/PU/newres/New' # publish directory

#Experiment Number:
expN = 1 # change this every time you want to run new feature. 

from pathlib import Path

import sys
sys.path.append('../nTuplizer')

from allInfo import GetAllRuns, GetAllEras

def export_graph(g , n , pname, pdir):
    ghtml = g.to_html()
    Path('{0}/{1}/'.format(pdir , pname) ).mkdir(parents=True, exist_ok=True)
    with open('{0}/{1}/{2}.html'.format(pdir , pname , n) , 'w') as f:
        f.write(ghtml)
def publish_res(rinfo , gchi2 , vname ,  name , publish_dir):

    
    
    
    gg = rinfo.plot_lumi_distribution(subRuns=-1 , colorLumiScale=4 , density=False)
    
    # updated plot layout in Class1
#     gg.update_layout(
#         title="Distribution of BX luminosity",
#         xaxis_title='luminosity (1/mb 1/25ns)',
#         yaxis_title="probability",
#         legend_title="Runs",
#     )
    export_graph(gg , '01-lumi_distribution' , name , publish_dir)
    
    
    
    
    #02-pu_distribution
    
    gg = rinfo.plotPUDists(70)
    export_graph(gg , '02-pu_distribution' , name , publish_dir)
    
    
    
    
    
    #03-{vname}_distribution:
    
    if rinfo.parentRun is None:
        gg = rinfo.plotDataDist(zoom=True)
        export_graph(gg , '03-{0}_distribution'.format(vname) , name , publish_dir) 

    
    
    
    #04-{vname}_predictions:

    gg = rinfo.plotPredictions([60,65,75,80,90])
    export_graph(gg , '04-{0}_predictions'.format(vname) , name , publish_dir)
    
        
        
        
        
    for _xsec in [60,65,75,80,90]:
        gg = rinfo.plotRunPredictions(_xsec)
        export_graph(gg , '05-{0}_predictions_xsec{1}'.format(vname , _xsec) , name , publish_dir)
            
    
    
    export_graph(gchi2 , '06-bestfit_details'  , name , publish_dir)
    export_graph(rinfo.postFitPlots(), '07-postFit_plots'  , name , publish_dir)
    export_graph(rinfo.pullPlots(50 , 1.3) , '08-pullPlots' , name , publish_dir)
    export_graph(rinfo.NadjiehPullPlots(50 , 1.3) , '08-NadjiehPullPlots' , name , publish_dir)
    
    
    
    
    
    if rinfo.parentRun is None:
        export_graph(rinfo.aggregateFitRes() , '09-summary1' , name ,publish_dir)
        export_graph(rinfo.aggregateFitRes2() , '09-summary2' , name ,publish_dir)
        export_graph(rinfo.aggregateFitRes3() , '09-summary3' , name ,publish_dir)
        export_graph(rinfo.aggregateFitRes4() , '09-summary4' , name ,publish_dir)
        
        
   
rinfo = None
        
def process_and_publish(year , era , vname , vmin , vmax , vnbins , pu_max ,
                        lumiName = 'PHYSICSDel' , 
#                         lumiQ =  np.array([0,0.2,0.6,1.0]) ,
                        lumiQ =  np.array([0,1.0]) ,
                        lnbins_per_q = 100 ,
                        xsec = np.arange(30,100,1), nthreads = 30 , publish_dir = web2 ):
    nbins = lumiQ.size -1
  #  print(nbins)  
    
    #Runs
    allRuns = sorted( list(set( GetAllRuns(year , era) ) ) )
    Runs = allRuns[0:1]
    if len(Runs) == 1:
        ind = allRuns.index(Runs[0])
        runName = 'RunNo:{0}'.format(ind+1)
    else:
        runName = 'nRuns={0}'.format(len(Runs))
    runName
    
    
    
    head = 'expe={0}_{1}{2}_nLumiBins={3}_{4}'.format(expN,era, year,nbins , runName)
    name = '{0}/{1}'.format(head, vname)
    
    
    #00simulation plot:
    
    var_bins = np.linspace(vmin,vmax,vnbins+1)
    simDist = SimulationVSPu(vname , year = year , var_bins=var_bins , pu_max=pu_max , nthreads=nthreads)


    g = simDist.plot(param=1 , label = 'PU = 1')
    for i in range(10,90+1,10):
        theName = 'PU = {0}'.format(i)
        simDist.plot(param=i , g=g , 
                     scatterOpts={ 'name':theName}
                    )

    g.update_layout(
        title="Simulated distribution for PU ",
        xaxis_title= vname,
        yaxis_title="probability",
        legend_title="PU number",
    )    
    export_graph(g , '00-Simulation' , name , publish_dir )
    
    
    
    

    
    
    
    print('is going to run over {0} runs'.format( len(Runs)) )
    print('Number of Lumi Bins = {0}'.format(nbins))

    
    rinfo = RunInfo(0 , vname , var_bins , lumiName ,lumiQ , nbins_perq=lnbins_per_q , sub_runs=Runs , 
                    nthreads=nthreads , xsecs = xsec)
    
    rinfo.setSimulation(simDist)
    gchi2 = rinfo.fit()
    
#     print(vname)
#     print(var_bins)
#     print(lumiName)
#     print(lumiQ)
#     print(lnbins_per_q)
    
    try:
        
        publish_res(rinfo , gchi2 , vname , name , publish_dir)
        for sr in rinfo._subRuns : #+ rinfo._subRunsSameLumiBins:
            #ext = '/SRSimilarBinning' if sr._isSecondHand else '/SRSameBinning'
            publish_res(sr , None , vname , vname , '{0}/{1}/Runs/Run{2}'.format(publish_dir , head , sr.run) )
    except Exception as e:
        print(e)
        return head, rinfo

    #del rinfo
    #del simDist
    

    return head, rinfo
 

    

In [ ]:
variables = { 
              "nVertices" : ( "nVertices" , 90 , 0 , 90 ) ,
              "nGoodVertices" : ("nGoodVertices", 80, 0 , 80) ,
              "nEles" : ("nEles" , 10 , 0 , 10 ) ,
              "nMus" : ("nMus" , 10 , 0 , 10 ),
              "nChargedHadrons" : ("nChargedHadrons" , 120 , 0 , 1200 ),
              "nLostTracks": ("nLostTracks" , 35 , 0 , 35 ),
              "nPhotons" : ("nPhotons" , 120 , 0 , 600 ),
              "nNeutralHadrons" : ("nNeutralHadrons" , 60 , 0 , 120 ),
              "fixedGridRhoAll" : ("fixedGridRhoAll" , 40 , 0 , 40 ),
              "fixedGridRhoFastjetAll" : ("fixedGridRhoFastjetAll" , 40 , 0 , 40 ),
              "fixedGridRhoFastjetAllCalo" : ("fixedGridRhoFastjetAllCalo" , 25 , 0 , 25 ),
              "fixedGridRhoFastjetCentral" : ("fixedGridRhoFastjetCentral" , 50 , 0 , 50 ),
              "fixedGridRhoFastjetCentralCalo" : ("fixedGridRhoFastjetCentralCalo" , 20 , 0 , 20 ),
              "fixedGridRhoFastjetCentralChargedPileUp" : ("fixedGridRhoFastjetCentralChargedPileUp" , 35 , 0 , 35 ),
              "fixedGridRhoFastjetCentralNeutral" : ("fixedGridRhoFastjetCentralNeutral" , 12 , 0 , 12 )
            }

vvv = {}
year , era = 2018 , 'B'
for v,vinfo in variables.items():
     head , vvv[v]  = process_and_publish(year , era , vinfo[0] , vinfo[2] , vinfo[3] , vinfo[1] , 90  )
        

  0%|          | 0/92 [00:00<?, ?it/s, SIMULATION]

is going to run over 1 runs
Number of Lumi Bins = 1


  0%|          | 0/7 [00:00<?, ?it/s, RUN 0]

  0%|          | 0/5 [00:00<?, ?it/s, RUN 317080]

  0%|          | 0/5 [00:00<?, ?it/s, RUN 317080]

...

In order to reference traces by row and column, you must first use plotly.tools.make_subplots to create the figure with a subplot grid.


  0%|          | 0/92 [00:00<?, ?it/s, SIMULATION]

is going to run over 1 runs
Number of Lumi Bins = 1


  0%|          | 0/7 [00:00<?, ?it/s, RUN 0]

  0%|          | 0/5 [00:00<?, ?it/s, RUN 317080]

## name of this Experiment: uses in naming folders

In [ ]:
head

## making directories for overviewPlots both in source and publish directories

In [ ]:
# overview for publishL
import os

directory = "{0}/overview".format(head)
  
# Parent Directory path
parent_dir = web2
  
# Path
path = os.path.join(parent_dir, directory)
  

try:
    os.makedirs(path)
except OSError as error:
    print(error)

In [ ]:
#overviewPlots for source
import os

directory1 = 'overviewPlots/{0}'.format(head)
  
# Parent Directory path
parent_dir1 = web1
  
# Path
path = os.path.join(parent_dir1, directory1)
  

try:
    os.makedirs(path)
except OSError as error:
    print(error)

## Luminosity and PileUp dists:

In [ ]:
def export_Graph(g , n , pdir):
    ghtml = g.to_html()
    Path('{0}/'.format(pdir ) )
    with open('{0}/{1}.html'.format(pdir , n) , 'w') as f:
        f.write(ghtml)

In [ ]:
G = vvv['nVertices'].lumi_distribution.plot()
export_Graph(G , 'LumiDist', '{0}/{1}/'.format(parent_dir,directory))
G


In [ ]:
G = vvv['nVertices'].plot_lumi_distribution()
export_Graph(G , 'LumiDistMain', '{0}/{1}/'.format(parent_dir,directory))
G


In [ ]:
xsec = 70
G = vvv['nVertices'].plotPUDists(xsec)
# vvv['nVertices'].plotDataDist()

export_Graph(G , 'PUDist({0})'.format(xsec) , '{0}/{1}/'.format(parent_dir,directory))
# export_Graph(G , 'PUDist({0})'.format(xsec) , '{0}/{1}/'.format(parent_dir1,directory1))

G

## Creating overview plot for all lumi bins:

In [ ]:
#total plots export

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


XsecBestTot = {}
keys = []
values = []
nVar = len(variables.keys())
print(nVar)
for key in vvv:
    for i in range(vvv[key].nLumiBins):
#         print(i)
        if i == 20 - 1:
            print(i)
print(vvv[key].nLumiBins)
for i in range(vvv[key].nLumiBins):
    for key in vvv:
#         keys.append('{0} for lumi bin = {1} '.format(vvv[key].vname , i+1))
        keys.append(vvv[key].vname)
        values.append(vvv[key].fitResults[i].bestFit)
#     print('values for bin = {0} is: {1}'.format(i+1 , values))
#     print('------------------------------------')
#     print('variables for bin = {0} is: {1}'.format(i+1 , keys))
#     print('------------------------------------')
    XsecBest= {}
    for q in range(len(keys)):
        XsecBest[keys[q]] = values[q]
    
#     print('Dict for bin = {0} is: {1}'.format(i+1 , XsecBest))
#     print('------------------------------------')




    names = list(XsecBest.keys())
    values = list(XsecBest.values())


    #         print('best fit for var = {0} in lumi bin = {1} is : {2} '.format(vvv[key].vname , i+1 , vvv[key].fitResults[i].bestFit))

    plt.plot(names , values ,'o-' , label='lumi bin = {0}'.format(i+1))
    plt.xticks(rotation=290)

    plt.xlabel("Variable")
    plt.ylabel("Xsection")
    plt.title("Best Xsection for different variables for different lumi bins for MCTune = 5")

    y_est = 69.2 #cross section
    y_err = 69.2*.046 #cross section error

    plt.plot(y_est, '-')
    plt.fill_between(names, y_est - y_err, y_est + y_err, alpha=0.2 , label = u'69.2 \u00B1 4.6%' )

    plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
#     plt.show()
    plt.savefig('{0}/{1}/totallumi.png'.format(parent_dir,directory) , dpi=300, bbox_inches='tight')   # save the figure to file
    plt.savefig('{0}/{1}/totallumi.png'.format(parent_dir1,directory1) , dpi=300, bbox_inches='tight')   # save the figure to file

    #     plt.close(fig)
    
#     fig.write_html('/plot for lumi =1')

#     plt.savefig('XSection Best fit for lumi = {0}.png'.format(i+1))
    keys = []
    values = []
# print(keys)
# print(values)
#     print(values)
for i in range(len(keys)):
    XsecBestTot[keys[i]] = values[i]

# print(XsecBestTot)


## Creating overview plot for each lumi bins:

In [ ]:
#plot for each lumi bin

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


XsecBestTot = {}
keys = []
values = []
nVar = len(variables.keys())
print(nVar)

for i in range(vvv[key].nLumiBins):
    for key in vvv:
#         keys.append('{0} for lumi bin = {1} '.format(vvv[key].vname , i+1))
        keys.append(vvv[key].vname)
        values.append(vvv[key].fitResults[i].bestFit)
#     print('values for bin = {0} is: {1}'.format(i+1 , values))
#     print('------------------------------------')
#     print('variables for bin = {0} is: {1}'.format(i+1 , keys))
#     print('------------------------------------')
    XsecBest= {}
    for q in range(len(keys)):
        XsecBest[keys[q]] = values[q]
    
#     print('Dict for bin = {0} is: {1}'.format(i+1 , XsecBest))
#     print('------------------------------------')




    names = list(XsecBest.keys())
    values = list(XsecBest.values())


    #         print('best fit for var = {0} in lumi bin = {1} is : {2} '.format(vvv[key].vname , i+1 , vvv[key].fitResults[i].bestFit))

    plt.plot(names , values ,'o-' , color='tab:brown' , label='lumi bin = {0}'.format(i+1))
    plt.xticks(rotation=290)

    plt.xlabel("Variable")
    plt.ylabel("Xsection")
    plt.title("Best Xsection for different variables for lumi bin = {0} (MCTune = 5)".format(i+1))

    y_est = 69.2 #cross section
    y_err = 69.2*.046 #cross section error

    plt.plot(y_est, '-')
    plt.fill_between(names, y_est - y_err, y_est + y_err, alpha=0.2 , label = u'69.2 \u00B1 4.6%' )

    plt.legend()
#     plt.savefig('{0}/{1}/plot_for_lumi_{2}.png'.format(parent_dir,directory,i+1) , dpi=300, bbox_inches='tight')
#     plt.savefig('{0}/{1}/plot_for_lumi_{2}.png'.format(parent_dir1,directory1,i+1) , dpi=300, bbox_inches='tight')   # save the figure to file

    plt.show()

    
#     fig.write_html('/plot for lumi =1')

#     plt.savefig('XSection Best fit for lumi = {0}.png'.format(i+1))
    keys = []
    values = []
# print(keys)
# print(values)
#     print(values)
for i in range(len(keys)):
    XsecBestTot[keys[i]] = values[i]

# print(XsecBestTot)


In [ ]:
#plot for each lumi bin

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


XsecBestTot = {}
keys = []
values = []
nVar = len(variables.keys())
print(nVar)

for i in range(vvv[key].nLumiBins):
    for key in vvv:
#         keys.append('{0} for lumi bin = {1} '.format(vvv[key].vname , i+1))
        keys.append(vvv[key].vname)
        values.append(vvv[key].fitResults[i].bestFit)
#     print('values for bin = {0} is: {1}'.format(i+1 , values))
#     print('------------------------------------')
#     print('variables for bin = {0} is: {1}'.format(i+1 , keys))
#     print('------------------------------------')
    XsecBest= {}
    for q in range(len(keys)):
        XsecBest[keys[q]] = values[q]
    
#     print('Dict for bin = {0} is: {1}'.format(i+1 , XsecBest))
#     print('------------------------------------')




    names = list(XsecBest.keys())
    values = list(XsecBest.values())


    #         print('best fit for var = {0} in lumi bin = {1} is : {2} '.format(vvv[key].vname , i+1 , vvv[key].fitResults[i].bestFit))

    plt.plot(names , values ,'o-' , color='tab:brown' , label='lumi bin = {0}'.format(i+1))
    plt.xticks(rotation=290)

    plt.xlabel("Variable")
    plt.ylabel("Xsection")
    plt.title("Best Xsection for different variables for lumi bin = {0} (MCTune = 5)".format(i+1))



    plt.legend()
#     plt.savefig('/eos/user/c/cmstandi/www/PU/newres/data2/5/plot for lumi = {}.png'.format(i+1) , dpi=300, bbox_inches='tight')
    plt.show()

    
#     fig.write_html('/plot for lumi =1')

#     plt.savefig('XSection Best fit for lumi = {0}.png'.format(i+1))
    keys = []
    values = []


## some features of the code:

In [ ]:
for key in vvv:
    for i in range(vvv[key].nLumiBins):
        print('best fit for var: {0} in lumi bin = {1} is : {2} '.format(vvv[key].vname , i+1 , vvv[key].fitResults[i].bestFit))

In [ ]:
vvv['nVertices'].subRunNumbers


In [ ]:
for i in  vvv['nVertices']._subRuns:
    x = i.run
    print(x)
print(len(vvv['nVertices']._subRuns))

In [ ]:
x = vvv['nVertices'].subRunNumbers
print(x)

In [ ]:

for key in vvv:
    for i in range(vvv[key].nLumiBins):
        print('best fit for var = {0} in lumi bin = {1} is : {2} '.format(vvv[key].vname , i+1 , vvv[key].fitResults[i].bestFit))

In [ ]:
for key in vvv:
    for i in range(vvv[key].nLumiBins):
        if i == 3 - 1:
            print('best fit for var = {0} in lumi bin = {1} is : {2} '.format(vvv[key].vname , i+1 , vvv[key].fitResults[i].bestFit))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


XsecBest = {}
keys = []
values = []
nVar = len(variables.keys())
print(nVar)

for i in range(vvv[key].nLumiBins):
    for key in vvv:
        keys.append(vvv[key].vname)
        values.append(vvv[key].fitResults[i].bestFit)

        
        
        
print(keys)
print(values)

In [ ]:
for i in range (vvv['nVertices'].nLumiBins):
    print(vvv['nVertices'].data_hists[i].integral())

In [ ]:
vvv['nVertices'].predictions

In [ ]:
vvv['nVertices'].plotPredictions([45])

In [ ]:
vvv['nVertices'].predictions[0].plot()

In [ ]:
 vvv['nVertices'].predictions[0].plot(param=vvv['nVertices'].fitResults[0].bestFit ,
                                    norm=vvv['nVertices'].data_hists[0].integral()
                                    )

In [ ]:
vvv['nVertices'].predictions[0].plot(norm=vvv['nVertices'].data_hists[0].integral() ,param=vvv['nVertices'].fitResults[0].bestFit)


In [ ]:

prelist = vvv['nVertices'].predictions[0].allProbs()
print(prelist)
print(sum(prelist))

In [ ]:
vvv['nVertices'].data_hists[0].plot()


In [ ]:
datalist =(vvv['nVertices'].data_hists[0].allProbs())/(vvv['nVertices'].data_hists[0].integral())
print(datalist)
print(sum(datalist))

In [ ]:
n = 0
for i in range(len(datalist)):
#     print(datalist[i])
#     print(prelist[i])
# #     print('-------')
    if datalist[i] == 0 :
        pass
    else: 
        m = datalist[i] - prelist[i]
        k = m**2
        j = k/datalist[i]
#         print(j)
        n += j
#         print(n)
        m = 0
#         print('-----------')
print(n/len(datalist))

In [ ]:
import math 
n = 0
for i in range(len(datalist)):
#     print(datalist[i])
#     print(prelist[i])
# #     print('-------')
    if datalist[i] == 0 :
        pass
    else: 
        m = prelist[i] - datalist[i]
#         j = m/math.sqrt(datalist[i])
        j = m/math.sqrt(datalist[i])
        n += j
        m = 0
print(n)

In [ ]:
 vvv['nVertices']._subRunsSameLumiBins[0].fitResults

In [ ]:
vvv['nVertices'].fitResults[0].plot()

In [ ]:
vvv['nVertices'].predictions[0].plot()

In [ ]:
vvv['nVertices'].data_hists[0].plot()

In [ ]:
print(vvv['nVertices'].nLumiBins)

In [ ]:
vvv['nVertices'].fitResults

In [ ]:
simDist = SimulationVSPu('nPhotons' , year = 2018 , var_bins= np.linspace(0,100,101) , pu_max=100 , nthreads=30)

In [ ]:
aa = simDist.plotEfficiencies()
aa.plot()

In [ ]:
for key in variables.keys():
    print('vName: {0}'.format(key))

In [ ]:
MC = SimulationVSPu.mctune
print(MC)

In [ ]:
# گرفتن انتگرال pu dist

# bina = binning(1 , 2 , 1.5 )
# binb = binning(2 , 3 , 2.5)
# binc = binning(3, 4 , 3.5)
# lumidist = lumiDistNumpy('testlumi' , {bina: 0.33 , binb: 0.33 , binc: 0.33 #lumi bin prob
#                                       }, 50 , np.array([1,2,2.5] # pu max and cross section for each bin 
#                                                       ))
# g = lumidist.PUDist.plot(param = 1)

# lumidist.PUDist.plot(param = 2 , g=g )
# lumidist.PUDist.plot(param = 3 , g=g )


# g.update_layout(
#         title="PU dist ",
#         xaxis_title= 'PU',
#         yaxis_title="probability",
#         legend_title="PU number",
#     )
# g

In [ ]:
# bina.representative

In [ ]:
# counter = 0
# for b,v in lumidist.PUDist.vals:
#     counter +=1
#     print(b,v)
# print(counter)